<a href="https://colab.research.google.com/github/SAMYSOSERIOUS/Best-Machine-Learning-Projects-for-Beginners/blob/main/14_Prediction_Miniapp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building a prediction "miniapp"

Assuming we have already successfully trained a model for image classification (or in our case someone else did on [ImageNet](https://image-net.org/) and made it available for download [here](https://www.tensorflow.org/api_docs/python/tf/keras/applications)), we would like to build up a minimalistic web based "application" (basically a webpage in static HTML) that can accept an image upload and return the top 3 predictions in a really "bare bones" format.

## Prerequisites

We will use the minimalist Python webserver [Flask](https://flask.palletsprojects.com/en/2.0.x/) to build our solution.

As for easy accessibility through Colab (aka. "someone else's computer") we use [Ngrok](https://ngrok.com/) to expose our work to be available from "outside" the Colab machine, that is, thorough our browser. Luckily, we don't have to deal to much with this, we just use the [Flask-Ngrok](https://github.com/gstaff/flask-ngrok) package, that handles the hassle for us. Only thing to keep in mind is, that if we run the notebook in Colab, the solution we build will __only be reachable via the ugly looking Ngrok link__, not the "localhost" or "127.0.0.1".

In [25]:
!pip install flask --quiet
!pip install flask-ngrok --quiet
!pip install pyngrok --quiet

ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of application context.

This typically means that you attempted to use functionality that needed
the current application. To solve this, set up an application context
with app.app_context(). See the documentation for more information.
ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/google

Fro image formatting we will use the [PIL](https://pillow.readthedocs.io/en/stable/) Python Imaging Library and [Numpy](https://numpy.org/) is always nice to have.

In [26]:
from PIL import Image
import numpy as np


## Downloading the model

In [27]:
from  tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import decode_predictions


In [28]:
model = VGG16()
# Luckily, the instantiation of a pretrained model obejct
# does a complete download in the background
# then initializes the model with the pretrained weights

## Image preprocessing and prediction functions


In [29]:
import sys, json

def load_image(filename):
    """Given a filename, assuming it is an image, we load it from disc.

    Parameters
    ----------
    filename : str
        Name of input file.

    Returns
    -------
    PIL Image object"""
    #TODO: use the Image class we imported from PIL to load the image.
    #Keep it simple!


    image = Image.open(filename)

    return image

In [30]:
def resize_image(image):
    """Given a PIL image object we resize it to fit into our downloaded model.

    Parameters
    ----------
    image : PIL Image object

    Returns
    -------
    PIL Image object"""
    model_accepted_shape = model.inputs[0].shape[1:3] #ugly magic to get input shape from model
    #TODO: please use the shape above to reshape the image with PIL and return the image object
    image = image.resize(model_accepted_shape)
    return image

In [31]:
def to_numpy(image):
    """Given a PIL image object we convert it to a Numpy array.

    Parameters
    ----------
    image : PIL Image object

    Returns
    -------
    Numpy.Ndarray"""
    array = np.array(image)
    #Observe: Numpy is intelligent enough to accept a PIL object as input
    #and convert it
    return array

In [32]:
def check_channels(array):
    """Given a numpy tensor we check for it's dimensionality,
    and if too many channels are there, we drop 1.
    This can be necessary in case of certain PNG images having additional channels.

    Parameters
    ----------
    array : Ndarray

    Returns
    -------
    Numpy.Ndarray"""
    if array.shape[2]>3:
        array = array[:,:,:3]
    #Ugly magic to drop a channel - the last one - in cease there are too many
    return array

In [33]:
def create_batch(array):
    """Given a numpy tensor we create a single element "batch" from it by adding a dimension.
    This is necessary because the model was trained with "minibatches" of data,
    so it assumes, that it does not get one single unput, but a bunch of them at once.

    Parameters
    ----------
    array : Ndarray

    Returns
    -------
    Numpy.Ndarray"""
    batch = np.expand_dims(array, axis=0)
    return batch

In [34]:
def execute_prediction(batch, model):
    """Given a numpy tensor representing the "batch" and the loaded model object,
    we execute the prediction.

    Parameters
    ----------
    batch : Ndarray,
    model : TF.Keras.model

    Returns
    -------
    prediction in Keras model's own format"""
    prediction = model.predict(batch)
    return prediction

In [35]:
def interpret_prediciton(prediction):
    """Given Keras model's prediction, we parse it and select the top 3.
    For this ve utilize the default `decode_predictions` function of TF.Keras
    For more info, see https://www.tensorflow.org/api_docs/python/tf/keras/applications/imagenet_utils/decode_predictions

    Parameters
    ----------
    array : Keras model's prediction

    Returns
    -------
    dict
        A dictionary containing the top 3 predictions."""
    label = decode_predictions(prediction)
    decoded_prediction = []
    # retrieve the most likely result, e.g. highest probability
    for l in label[0][:3]:
        print('%s (%.2f%%)' % (l[1], round(l[2]*100,2)))
        #some nice formatting to dicts
        decoded_prediction.append({"class": l[1], "probability":round(l[2]*100,2)})
    return decoded_prediction

In [36]:
def human_format_prediction(decoded_prediction):
    """Given the processed prediction dict, we modify it to look nice to people

    Parameters
    ----------
    dict : Interpreted prediction top 3

    Returns
    -------
    str
        A string representing human readable output."""
    formatted_output = ""
    for element in decoded_prediction:
        formatted_output += str(element)
        formatted_output += "<br>" #some nice line breaks in HTML rendering
    return formatted_output

In [37]:
def json_format_prediction(decoded_prediction):
    """Given the processed prediction dict, we modify it to become proper JSON

    Parameters
    ----------
    dict : Interpreted prediction top 3

    Returns
    -------
    str
        A JSON conformant string representing of the output."""
    return json.dumps(decoded_prediction)

In [38]:
def do_prediction(filename):
    """Main prediction function.
    - Takes in a filename,
    - calls preprocessing steps,
    - returns formatted prediction.

    Parameters
    ----------
    filename : str
        Name of input file.

    Returns
    -------
    str
        A string representingoutput."""
    image = load_image(filename)
    image = resize_image(image)
    array = to_numpy(image)
    array = check_channels(array)
    batch = create_batch(array)
    prediction = execute_prediction(batch, model)
    decoded_prediction = interpret_prediciton(prediction)
    formatted_output = human_format_prediction(decoded_prediction)
    return formatted_output

Please observe, that the model will systemmatically err in the direction fo "slim" dog breeds, because the way we implemented "resize" of the original image - ie. we "squeeze" te image to conform to 224x224 pixels!

## Flask app

Since we have nice functions that can take in an image and return a prediction, we can now start to develop the mini "webapp".

### Basic HTML page

Let's first define a basic HTML page layout tha thas a single control element: an upload "form" which can take in files via the browser. This will be our way to feed in the inputs for prediction.

In [39]:
HTML = """
<!doctype html>
<html>
  <head>
    <title>File Upload</title>
  </head>
  <body>
    <h1>File Upload</h1>
    <form method="POST" action="" enctype="multipart/form-data">
      <p><input type="file" name="file"></p>
      <p><input type="submit" value="Submit"></p>
    </form>
  </body>
</html>"""

### "Main app"

Now we define a minimalistic Flask app, that defines single page endpoint at "/", so the root URL.

If a HTTP ("GET") request comes in to this URL, the app will automatically respond by "serving" the HTML content we difend above, so the requesting browser can render the "webpage" we built.

After this the user is free to interact with our wonderful form to choose an appropriate image file. We are assuming here that the user acts non-maliciously, so we do not do all the checks for file content and structure that would protect us, just assume, that the user uploads well behaving image files.

When the user pushes "Submit", this automatically generates a "POST" HTTP request towards our Flask app's "/" or "root" endpoint and with it, it "posts" or sends the image file, which we handle by defining an `upload_file` endpoint, that saves the file and calls on it the `do_prediction` function we defined above. (Please observe, that we don't care about the saved files, so they might accumuate, collide...)

In [40]:
from flask import *

app = Flask(__name__)

#TODO Use the Flask guide to define the simplest possible endpoint for the "root url"
@app.route('/')
def root():
    return HTML

#TODO Use the Flask guide to define a POST endpoint for the "root url"
@app.route('/', methods=['POST'])
def upload_file():
    uploaded_file = request.files['file']
    if uploaded_file.filename != '':
        uploaded_file.save(uploaded_file.filename)

    result = do_prediction(uploaded_file.filename)

    return result

And finally we start running our "web application" via the Ngrok tunnel.

In [41]:
from pyngrok import ngrok
import threading
import time

# Terminate ngrok tunnels if any are already running
ngrok.kill()

# Set the ngrok authtoken directly
ngrok.set_auth_token("put ngrok token here")

# Start ngrok tunnel on port 5000 (Flask's default port)
public_url = ngrok.connect(5000).public_url
print(f" * ngrok tunnel available at: {public_url}")

# Create a thread to run the Flask app
# This prevents app.run() from blocking the notebook
def run_flask_app():
    app.run(debug=False, use_reloader=False)

flask_thread = threading.Thread(target=run_flask_app)
flask_thread.daemon = True
flask_thread.start()

# Wait a bit for the Flask app to start before the notebook cell finishes
time.sleep(2)

 * ngrok tunnel available at: https://a737b8ebcafa.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


In case local execution fails, [this](https://githubmemory.com/repo/gstaff/flask-ngrok/issues/20) might help.

# Testing

Please open the temporary ngrok.io link that is visible in the cell output, grab a pictrure from somewhere (eg. an image of a dog) and use the submit functionality!

After a single prediction, you can get back to the main form with the back button.